# Unauthorized Access to Docker Socket - Investigation and Response
This notebook is designed to investigate unauthorized or suspicious access to the Docker socket within your AWS environment. We will query CloudTrail logs using Athena to detect and analyze such activities.

We are following best practices from the AWS Incident Response guide:

https://docs.aws.amazon.com/whitepapers/latest/aws-security-incident-response-guide/aws-security-incident-response-guide.html

# Alerting and Detection Strategy (ADS)
## Goal
Detect unauthorized or suspicious access to the Docker socket, which could lead to privilege escalation.
## Categorization
MITRE ATT&CK: Privilege Escalation
## Strategy Abstract
This ADS uses AWS GuardDuty alerts and Athena queries to detect unauthorized Docker socket access. It also includes Python code for automated responses.
## Technical Context
AWS GuardDuty generates alerts for suspicious activities. Athena queries can further filter these alerts. Python code can automate responses.
## Blind Spots and Assumptions
Assumes GuardDuty and CloudTrail are properly configured. May not detect activities that bypass these services.
## False Positives
Administrative activities that require Docker socket access.
## Validation
Trigger a GuardDuty alert by accessing the Docker socket in a test environment.
## Priority
High
## Response
Isolate affected instance, investigate the root cause, and apply necessary patches.
## Additional Resources
AWS GuardDuty Documentation, AWS Athena Documentation

## Identification and Isolation
1. Open the GuardDuty console at https://console.aws.amazon.com/guardduty/.
2. On the Findings page, choose the corresponding finding to open the findings panel.
3. In the findings panel, under the Resource affected section, you can view the container's ID and Name.
4. Move to the setup section and run the Athena queries to be investigating the root cause.
5. Isolate this container from other container workloads if the alert is confirmed malicous. 

# Setup

## Load Libraries
To query CloudTrail and interact with AWS, we need to load several libraries and configure our environment.

In [ ]:
pip install pyathena --quiet

In [ ]:
import boto3
import pandas as pd
from pyathena import connect
region = 'us-east-1'
athena_bucket = 's3:// ENTER BUCKET NAME HERE /'
db_name = 'security_analysis'

## 1.0 Set up helper function for Athena

The Python query_results function will help you query Athena tables.

In [ ]:
def query_results(sql):
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    columns = cursor.description
    data = cursor.fetchall()
    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]
    df = pd.DataFrame(rows, columns=column_names)
    df1 = df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    return df1.set_properties(**{'text-align': 'center'})

# 2.0 Docker Socket Investigation

### 2.1 Investigating Unauthorized Access
Lists unauthorized access attempts to the Docker socket with additional information such as the user identity ARN, time, AWS service, API call, region, and source IP address.

In [ ]:
unauthorized_access = f"""
SELECT *
FROM \"{db_name}\".\"cloudtrail\"
WHERE eventSource = 'ecs.amazonaws.com'
AND eventName = 'CreateContainerInstance'
AND requestParameters LIKE '%docker.sock%'
ORDER BY eventTime DESC
LIMIT 10
"""

In [ ]:
results = query_results(unauthorized_access)
results

### 2.2 Additional Athena Queries for Investigation
You can perform additional Athena queries to further investigate the unauthorized or suspicious access to the Docker socket.

In [ ]:
additional_query = f"""
SELECT userIdentity.arn, count(*) AS total
FROM \"{db_name}\".\"cloudtrail\"
WHERE eventSource = 'ecs.amazonaws.com'
AND eventName = 'CreateContainerInstance'
AND requestParameters LIKE '%docker.sock%'
GROUP BY userIdentity.arn
ORDER BY total DESC
"""

In [ ]:
results = query_results(additional_query)
results

### 3.0 Automated Response

## Pause the Container
Suspend all the processes in your container.

In [ ]:
import docker
client = docker.from_env()

def pause_container(container_id):
    container = client.containers.get(container_id)
    container.pause()
    print(f'Container {container_id} has been paused.')

# Uncomment the next line and replace 'container_id' with the actual container ID to execute the pause
# pause_container('container_id')

## Stop the Container
If pausing the container fails, stop the container from running.

In [ ]:
def stop_container(container_id):
    container = client.containers.get(container_id)
    container.stop()
    print(f'Container {container_id} has been stopped.')

# Uncomment the next line and replace 'container_id' with the actual container ID to execute the stop
# stop_container('container_id')

## Evaluate the Presence of Malware
Evaluate if malware was in the container's image. Consider scanning the image with AWS Guard Duty.
https://docs.aws.amazon.com/guardduty/latest/ug/malware-protection.html